In [ ]:
!sudo apt install libblas-dev llvm python3-pip python3-scipy
!virtualenv --system-site-packages -p python3 env
!source env/bin/activate
!pip install llvmlite==0.15.0
!pip install numba==0.30.1
!pip install librosa

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binfmt-support libffi-dev libimagequant0 liblbfgsb0 libpfm4 libraqm0
  libxsimd-dev libz3-4 libz3-dev llvm-14 llvm-14-dev llvm-14-runtime
  llvm-14-tools llvm-runtime mailcap mime-support python3-beniget
  python3-decorator python3-gast python3-numpy python3-olefile python3-pil
  python3-ply python3-pygments python3-pythran python3-setuptools
  python3-wheel python3-yaml
Suggested packages:
  liblapack-doc libxsimd-doc llvm-14-doc python-numpy-doc python3-pytest
  python-pil-doc python-ply-doc python-pygments-doc ttf-bitstream-vera
  python-scipy-doc python-setuptools-doc
The following NEW packages will be installed:
  binfmt-support libblas-dev libffi-dev libimagequant0 liblbfgsb0 libpfm4
  libraqm0 libxsimd-dev libz3-4 libz3-dev llvm llvm-14 llvm-14-dev
  llvm-14-runtime llvm-14-tools llvm-runtime mailcap mime-support
  python3-benige

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

import subprocess
import shlex

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-randr0 libxcb-render-util0 libxcb-util1 libxcb-xinerama0
  libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme
  qttranslations5-l10n timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common 

In [ ]:
!pip install utils
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=cad90a69d85db7847a2f923840aa2668b478657475de91bebfe192ae35e30a2c
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [ ]:
!pip install pydub

In [ ]:
import os
import music21 as m21
import torch
import librosa
import utils
import pretty_midi
import torch.nn as nn
import numpy as np
from collections import defaultdict
from torch.nn.utils.rnn import pad_sequence
import subprocess
import shlex
import random

random.seed(1)


one hot encoding of ground truth labels

In [ ]:
#use one-hot encoding to represent the ground truth labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#sample music genre labels
genres = ['Reggae', 'Rap', 'Electronic', 'Jazz', 'Latin', 'Blues', 'Ambient', 'Pop', 'Classical', 'Soul', 'Country', 'Rock']

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the LabelEncoder to the genre labels and transform them to numeric values
encoded_genres = label_encoder.fit_transform(genres)

print(encoded_genres)  # Output: [ 9  8  4  5  6  1  0  7  2 11  3 10]

# Reshape the encoded genre labels to a 2D array (required for OneHotEncoder)
reshaped_genres = encoded_genres.reshape(-1, 1)

# Create a OneHotEncoder object
onehot_encoder = OneHotEncoder()

# Fit the OneHotEncoder to the reshaped genre labels and transform them to one-hot encoded vectors
onehot_encoded_genres = onehot_encoder.fit_transform(reshaped_genres).toarray()

one_hot_dict = {}
for i in range(len(genres)):
  one_hot_dict[genres[i]] = onehot_encoded_genres[i]

print(one_hot_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

[ 9  8  4  5  6  1  0  7  2 11  3 10]
{'Reggae': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]), 'Rap': array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), 'Electronic': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), 'Jazz': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), 'Latin': array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), 'Blues': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Ambient': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Pop': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]), 'Classical': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Soul': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]), 'Country': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Rock': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])}
cuda


In [ ]:
class ClassifierLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes = 12, dropout_prob = 0.5):
    super(ClassifierLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    #lstm layer
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True)
    #intermediate fully connected layer to match output size
    self.fc_intermediate = nn.Linear(2 * hidden_size, hidden_size)
    #dropout layer to improve generalization
    self.dropout = nn.Dropout(dropout_prob)
    #fully connected layer
    self.fc = nn.Linear(hidden_size, num_classes)

    # Move all parameter tensors to the desired device (GPU if available)
    self.to(torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

  def forward(self, x):
    #ensure the input is a Pytorch tensor
    x = torch.tensor(x, dtype = torch.float32)
    # Move the input tensor to the GPU (cuda:0)
    x = x.to(device)

    x = x.unsqueeze(0) #Add a batch dimension
    #Set the initial hidden states
    h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device).to(x.dtype) #ensure that the model and data are on the same device (GPU)
    #initialize cell state
    c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device).to(x.dtype) #ensure that the model and data are on the same device (GPU)
    #forward propagate the lstm
    out, _ = self.lstm(x, (h0, c0))
    #apply fully connected layer to match output size
    out = self.fc_intermediate(out)
    # Get the last time step's output
    out = out[:, -1, :]
    #Apply dropout to the LSTM output
    out = self.dropout(out)
    #Decode the hidden state of the last time step
    out = self.fc(out)
    # Apply softmax activation to obtain class probabilities
    out = torch.softmax(out, dim=1)
    return out

class GeneratorLSTMModule(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(GeneratorLSTMModule, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Check if GPU is available
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.ident = torch.eye(input_size).to(self.device)  # Make sure identity matrix is on the correct device
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True).to(self.device)  # Send module to correct device
        self.dropout = nn.Dropout(dropout_prob).to(self.device)  # Send module to correct device
        self.fc = nn.Linear(hidden_size, output_size).to(self.device)  # Send module to correct device

    def forward(self, x):
        x = x.to(self.device)  # Make sure input tensor is on the correct device
        one_hot_list = [self.ident[t.long()] for t in x]
        one_hot = torch.stack(one_hot_list)
        one_hot = one_hot.view(x.shape[0], -1, self.input_size)
        _, (hidden, _) = self.lstm(one_hot)
        hidden = self.dropout(hidden[-1])
        output = self.fc(hidden)
        return output

Data processing

In [ ]:
def convert_midi_to_wav(midi_file_path, output_file_path, soundfont_path = shlex.quote('font.sf2')):
    command = [
        "fluidsynth",
        "-ni",
        soundfont_path,
        midi_file_path,
        "-F",
        output_file_path,
        "-r",
        "44100"
    ]
    subprocess.run(command)

from pydub import AudioSegment
def detect_leading_silence(audio_segment, silence_threshold=-50.0, chunk_size=10):
    trim_ms = 0  # Duration of leading silence in milliseconds
    silence_threshold = audio_segment.dBFS + silence_threshold

    for i in range(len(audio_segment)//chunk_size):
        if audio_segment[i*chunk_size:(i+1)*chunk_size].dBFS < silence_threshold:
            trim_ms += chunk_size
        else:
            break

    return trim_ms
def detect_trailing_silence(audio_segment, silence_threshold=-50.0, chunk_size=10):
    trim_ms = 0  # Duration of trailing silence in milliseconds
    silence_threshold = audio_segment.dBFS + silence_threshold

    for i in range(len(audio_segment)//chunk_size, 0, -1):
        if audio_segment[(i-1)*chunk_size:i*chunk_size].dBFS < silence_threshold:
            trim_ms += chunk_size
        else:
            break

    return trim_ms

def slice_sound(input_file_path, output_file_path):
    audio = AudioSegment.from_file(input_file_path, format="wav")

    # Detect leading and trailing silence in the audio
    leading_silence = detect_leading_silence(audio)
    trailing_silence = detect_trailing_silence(audio)

    # Slice the audio without the leading and trailing silence
    sound_audio = audio[leading_silence:len(audio)-trailing_silence]

    # If the duration is less than 5 seconds, add silence to make it 5 seconds
    if len(sound_audio) < 5000:  # duration in milliseconds
        silence = AudioSegment.silent(duration=5000 - len(sound_audio))
        sound_audio += silence
    # If the duration is more than 20 seconds, take only the first 20 seconds
    elif len(sound_audio) > 20000:  # duration in milliseconds
        sound_audio = sound_audio[:20000]

    sound_audio.export(output_file_path, format="wav")

import wave

def get_wav_duration_in_seconds(file_path):
    with wave.open(file_path, 'rb') as wav_file:
        frames = wav_file.getnframes()
        frame_rate = wav_file.getframerate()
        duration_seconds = int(frames / frame_rate)
    return duration_seconds
def data_processing(file_path, num_mfcc = 13, n_fft = 2048, hop_length = 512, num_segment = 10):

  sample_rate = 44100 #each sample has a 44100 Hz sample rate
  track_duration = get_wav_duration_in_seconds(file_path) #measured in seconds
  samples_per_seg = int(sample_rate*track_duration/num_segment)

  try:
    #the load attribute of librosa returns a tuple of (ndarray, float),
    #where the ndarray represents audio time series, and sr represents the sampling rate of y
    signal, sr = librosa.load(file_path, sr = sample_rate)
    #print(signal)

    for n in range(num_segment):
      mfcc = librosa.feature.mfcc(y = signal[(samples_per_seg * n): (samples_per_seg * (n+1))],
                                  sr = sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                  hop_length = hop_length)
      mfcc = mfcc.T #[216, 13]
      #pad the first dimension

      print(mfcc.shape)
      if mfcc.shape[0] != 1: #indicates signal of length > 0

        if mfcc.shape[0] > 516:   #longer than 60 seconds
          return None

        else:

          # Desired length to pad the first dimension
          desired_length = 516

          # Calculate the amount of padding required for the first dimension
          padding_needed = desired_length - mfcc.shape[0]

          # Pad the first dimension with zeros to achieve the desired length
          padded_array = np.pad(mfcc, ((0, padding_needed), (0, 0)), mode='constant')
          print(padded_array)
          print(padded_array.shape)
          return padded_array.tolist()

  except:
    return None

####This has been modified, check whether that's correct
###

def classifier(audio_file, model, one_hot_dict):
  #convert audio_file from midi.format to wav.format
  output_file_path = os.path.splitext(audio_file)[0] + ".wav"
  convert_midi_to_wav(audio_file,output_file_path)
  slice_sound(output_file_path, output_file_path)

  #extract mfcc feature from the file

  data = np.array(data_processing(output_file_path))

  #convert it to Pytorch tensor (Modified Aug 2nd, 2023)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  data_tensor = torch.tensor(data, dtype=torch.float32).to(device)

  # Perform prediction using the RNN model.
  predicted_genre_probabilities = model(data)

  # Get the index of the predicted genre (class with the highest probability).  (Modified Aug 2nd, 2023)
  predicted_genre_index = torch.argmax(predicted_genre_probabilities.cpu()).item()

  #get the predicted genre label in one_hot encoded form
  predicted_genre_one_hot = onehot_encoded_genres[predicted_genre_index]

  for genre in one_hot_dict:
      one_hot = one_hot_dict[genre]
      if np.array_equal(one_hot, predicted_genre_one_hot):
          output = genre
          break

  return output


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
classifier_model = ClassifierLSTM(input_size=13, hidden_size=64, num_layers=2, num_classes=12, dropout_prob=0)
classifier_model.to(device)
#classifier_model.load_state_dict(torch.load('classifier.pth'))
classifier_model.load_state_dict(torch.load('/content/gdrive/MyDrive/APS360/Music_gpt/classifier.pth'))
classifier_model.eval()
#echo's specific version
input = '/content/gdrive/MyDrive/APS360/Music_gpt/demo_bad.mid'
genre = classifier(input, classifier_model, one_hot_dict)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
(112, 13)
[[-876.8557      11.599958    11.436052  ...    7.311838     6.653559
     6.013711 ]
 [-880.6896       6.2270107    6.208866  ...    5.6583376    5.5454035
     5.424671 ]
 [-882.8333       3.2011888    3.2004652 ...    3.1773531    3.1723053
     3.1667807]
 ...
 [   0.           0.           0.        ...    0.           0.
     0.       ]
 [   0.           0.           0.        ...    0.           0.
     0.       ]
 [   0.           0.           0.        ...    0.           0.
     0.       ]]
(516, 13)


In [ ]:
print(genre)

Country


In [ ]:
def tokenize_score(score, time_step=0.25, input_size=66):
    tokens = []
    for part in score.parts:
        part_stream = part.flatten().notesAndRests
        prev_event = None
        symbol = -1  # Default value for 'symbol' variable

        for event in part_stream:
            if isinstance(event, m21.note.Note):
                symbol = event.pitch.midi
            elif isinstance(event, m21.note.Rest):
                symbol = -1  # Use -1 to represent rests

            steps = int(event.duration.quarterLength / time_step)
            tokens.extend([symbol] + [-1] * (steps - 1))

    # Pad the tokens to match the input_size
    if len(tokens) < input_size:
        tokens.extend([-1] * (input_size - len(tokens)))
    else:
        tokens = tokens[:input_size]

    return tokens


import music21 as m21
import torch


from music21 import midi

def generate_melody(model, input_midi_path, output_length=100, temperature=4.0):
    # Load the MIDI file and tokenize it
    input_tokens = tokenize_score(m21.converter.parse(input_midi_path))

    # Convert the tokens to a tensor and add an extra dimension for batch size
    input_tensor = torch.tensor(input_tokens).unsqueeze(0)

    input_tensor = input_tensor.to(model.device)

    # Initialize an empty list to store the output tokens
    output_tokens = []

    # Feed the input to the model and generate the output tokens
    with torch.no_grad():
        for _ in range(output_length):
            # Forward pass through the model
            output = model(input_tensor)

            # Apply temperature to the output distribution and sample a token
            output = output / temperature
            probabilities = torch.nn.functional.softmax(output, dim=1)
            token = torch.multinomial(probabilities, num_samples=1)

            # Append the token to the output list and update the input tensor
            output_tokens.append(token.item())
            input_tensor = torch.cat([input_tensor, token], dim=1)

    # Convert the output tokens back to MIDI notes
    output_score = m21.stream.Stream()
    for token in output_tokens:
        if token == -1:
            output_score.append(m21.note.Rest(quarterLength=0.25))
        else:
            output_score.append(m21.note.Note(midi=token, quarterLength=0.25))

    return output_score

#define input_size and output_size as element mapping for each genre
element_mapping = {'Ambient':74,
                   'Blues':63,
                   'Classical':79,
                   'Country':57,
                   'Jazz':59,
                   'Soul':62,
                   'Pop':57,
                   'Electronic':63,
                   'Rock':84,
                   'Rap':66,
                   'Reggae':54,
                   'Latin':69}

#input_size = element_mapping[genre]
input_size = 66
hidden_size = 128
output_size = input_size
dropout_prob = 0

In [ ]:
generator_model = GeneratorLSTMModule(input_size, hidden_size, output_size, dropout_prob).cuda()

#file_name = f"{genre}.pth"
file_name = '/content/gdrive/MyDrive/APS360/Music_gpt/rap.pth'
generator_model.load_state_dict(torch.load(file_name), strict = False) ##Modified Aug 2, 2023

generator_model.eval()  # Set the model to evaluation mod
output_score = generate_melody(generator_model, '/content/gdrive/MyDrive/APS360/Music_gpt/rap_test2.mid')
output_score.write('midi', fp='output.mid')  # Save the output as a MIDI file

'output.mid'